# Video Face Manipulation Detection Through Ensemble of CNNs

Code in this notebook is adapted from 
https://github.com/polimi-ispl/icpr2020dfdc

N. Bonettini, E. D. Cannas, S. Mandelli, L. Bondi, P. Bestagini and S. Tubaro, "Video Face Manipulation Detection Through Ensemble of CNNs," 2020 25th International Conference on Pattern Recognition (ICPR), 2021, pp. 5012-5019, doi: 10.1109/ICPR48806.2021.9412711.


In [1]:
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null
%cd icpr2020dfdc/notebook

Cloning into 'icpr2020dfdc'...
remote: Enumerating objects: 645, done.
remote: Counting objects: 100% (108/108), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 645 (delta 96), reused 81 (delta 81), pack-reused 537
Receiving objects: 100% (645/645), 99.63 MiB | 18.22 MiB/s, done.
Resolving deltas: 100% (336/336), done.
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=5f46c06c43b3dbf43b0eb5fc83171a3ca196256fc1e3909a62b559c52261a4ae
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-dzwr0dso
/content/icpr2020dfdc/notebook


In [6]:
!pip uninstall opencv-python-headless==4.5.5.62
!pip install opencv-python-headless==4.1.2.30

Found existing installation: opencv-python-headless 4.5.5.64
Uninstalling opencv-python-headless-4.5.5.64:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/cv2/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless-4.5.5.64.dist-info/*
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavcodec-65fa80df.so.58.134.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavformat-8ef5c7db.so.58.76.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libavutil-9c768859.so.56.70.100
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libbz2-a273e504.so.1.0.6
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libcrypto-09fe7800.so.1.1
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libgfortran-91cc3cb1.so.3.0.0
    /usr/local/lib/python3.7/dist-packages/opencv_python_headless.libs/libopenblas-r0-f650aae0.3.3.so
    /usr/local/lib/python3.7/dis

In [7]:
import torch
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
from scipy.special import expit
import pandas as pd

import sys
sys.path.append('..')
import os 


from blazeface import FaceExtractor, BlazeFace, VideoReader
from architectures import fornet,weights
from isplutils import utils

## Parameters

In [44]:
"""
Choose an architecture between
- EfficientNetB4
- EfficientNetB4ST
- EfficientNetAutoAttB4
- EfficientNetAutoAttB4ST
- Xception
"""
net_model = 'EfficientNetB4ST'

"""
Choose a training dataset between
- DFDC
- FFPP
"""
train_db = 'DFDC'

In [45]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224
frames_per_video = 30

## Initialization

In [46]:
model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))

Loaded pretrained weights for efficientnet-b4


Downloading: "https://f002.backblazeb2.com/file/icpr2020/EfficientNetB4ST_DFDC_bestval-86f0a0701b18694dfb5e7837bd09fa8e48a5146c193227edccf59f1b038181c6.pth" to /root/.cache/torch/hub/checkpoints/EfficientNetB4ST_DFDC_bestval-86f0a0701b18694dfb5e7837bd09fa8e48a5146c193227edccf59f1b038181c6.pth


  0%|          | 0.00/33.9M [00:00<?, ?B/s]

<All keys matched successfully>

In [47]:
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

In [48]:
facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
videoreader = VideoReader(verbose=False)
video_read_fn = lambda x: videoreader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn=video_read_fn,facedet=facedet)

## Get Predictions

In [49]:
model_name = net_model + '_' + train_db
result_folder = './Res_' + model_name

In [50]:
main_folder = './RealVideo-RealAudio'
label = 0 # change this according to the folder - 0 for real and 1 for fake
for ethnicity in os.listdir(main_folder):
  if 'DS_Store' in ethnicity:
    continue
  for gender in os.listdir(os.path.join(main_folder,ethnicity)):
    if 'DS_Store' in gender:
      continue

    paths = []
    inputs = []
    for video_id in os.listdir(os.path.join(main_folder,ethnicity,gender)):
      if 'DS_Store' in video_id:
        continue
      for file_name in os.listdir(os.path.join(main_folder,ethnicity,gender,video_id)):
        if file_name.endswith('.mp4'):
          video_path = os.path.join(main_folder,ethnicity,gender,video_id,file_name)
          frames = face_extractor.process_video(video_path)
          faces = torch.stack( [ transf(image=frame['faces'][0])['image'] for frame in frames if len(frame['faces'])] ) # num_frames x 3 x 224 x 224
          inputs.append(faces)
          paths.append(video_path)

    # concat inputs 
    preds = []
    for video_input in inputs:
      with torch.no_grad():
        video_pred = net(video_input.to(device)).cpu().numpy().flatten()
        video_pred = expit(video_pred.mean())
        preds.append(video_pred)

    # make diretory to store result
    result_path_dir = os.path.join(result_folder,ethnicity,gender)
    if not os.path.exists(result_path_dir):
      os.makedirs(result_path_dir)
    
    # store result in a csv
    df = pd.DataFrame({
        'path': paths,
        'ethnicity': [ethnicity] * len(paths),
        'gender': [gender] * len(paths),
        'pred': preds,
        'label': [label] * len(paths)
    })

    df.to_csv(os.path.join(result_path_dir,'results.csv'),index=False)

    print('Done with ' + ethnicity + ' ' + gender)


print('done')


Done with African men
Done with African women
Done with Caucasian (American) men
Done with Caucasian (American) women
Done with Caucasian (European) men
Done with Caucasian (European) women
Done with Asian (South) men
Done with Asian (South) women
Done with Asian (East) men
Done with Asian (East) women
done


In [51]:
# create a zip of the result folder and download it
!zip -r Res_EfficientNetB4ST_DFDC.zip Res_EfficientNetB4ST_DFDC/

  adding: Res_EfficientNetB4ST_DFDC/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/African/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/African/men/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/African/men/results.csv (deflated 82%)
  adding: Res_EfficientNetB4ST_DFDC/African/women/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/African/women/results.csv (deflated 82%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (American)/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (American)/men/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (American)/men/results.csv (deflated 86%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (American)/women/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (American)/women/results.csv (deflated 87%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (European)/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (European)/men/ (stored 0%)
  adding: Res_EfficientNetB4ST_DFDC/Caucasian (European)/men/results.csv (de